<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 15: Deep Learning Regression</h1>


<a name='section_15_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L15.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_1">L15.1 Discovering the Higgs with Deep Learning</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_15_1">L15.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_2">L15.2 Minimizing Loss with a Neural Network</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_15_2">L15.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_3">L15.3 An Example with PyTorch: Fitting a Parabola</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_15_3">L15.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_4">L15.4 Another Example: Fitting a Sine Function</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_15_4">L15.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_5">L15.5 Sine Function Continued: Adjusting the Network</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_15_5">L15.5 Exercises</a></td>
    </tr>
</table>

In [ ]:
#>>>RUN: L15.0-runcell00

!pip install torch
!pip install imageio
!pip install awkward
!pip install george
!pip install uproot
!pip install pylorentz

In [ ]:
#>>>RUN: L15.0-runcell01

import torch                        #https://pytorch.org/docs/stable/torch.html
import torch.nn as nn               #https://pytorch.org/docs/stable/nn.html
from torch.autograd import Variable #https://pytorch.org/docs/stable/autograd.html
import torch.nn.functional as F     #https://pytorch.org/docs/stable/nn.functional.html
import torch.utils.data as Data     #https://pytorch.org/docs/stable/data.html

import matplotlib.pyplot as plt     #https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

import numpy as np                  #https://numpy.org/doc/stable/
import imageio                      #https://imageio.readthedocs.io/en/stable/
import george                       #https://george.readthedocs.io/en/latest/
from george import kernels          #https://george.readthedocs.io/en/latest/user/kernels/

In [ ]:
#>>>RUN: L15.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_15_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L15.1 Discovering the Higgs with Deep Learning</h2>  

| [Top](#section_15_0) | [Previous Section](#section_15_0) | [Exercises](#exercises_15_1) | [Next Section](#section_15_2) |


In [ ]:
#>>>RUN: L15.1-slides


from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L19/slides_L19_01.html', width=970, height=550)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.1.1</span>

Look again at the Higgs discovery plots. At the Higgs mass, the CMS experiment is about 30% more sensitive than ATLAS, but both experiments found approximately the same excess. This is because:

A) The ATLAS experiment was less sensitive, but had more data.\
B) The ATLAS experiment had more advanced machine learning analysis tools.\
C) This was just a random fluctuation, and not a particularly unlikely one.\
D) The CMS data was noisier.

<a name='section_15_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L15.2 Minimizing Loss with a Neural Network</h2>  

| [Top](#section_15_0) | [Previous Section](#section_15_1) | [Exercises](#exercises_15_2) | [Next Section](#section_15_3) |


In [ ]:
#>>>RUN: L15.2-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L19/slides_L19_02.html', width=970, height=550)

In [ ]:
#>>>RUN: L15.2-runcell01

#Let's Try GP on a Gaussian random set of points
def gaussian(mu,sigma,norm,offset):
    """Returns a gaussian function with the given parameters"""
    return lambda x: norm*np.exp(-(1./2.)*((x-mu)/(sigma))**2)+offset

Xin = np.mgrid[0:201] # points from 0 to 201
data = gaussian(100., 20., 10., 5)(Xin) + 5*np.random.random(Xin.shape) # Guassian + semaring
plt.plot(data,"*")
plt.show()

In [ ]:
#>>>RUN: L15.2-runcell02

#now let's run GP on this guy

kernel = np.var(data) * kernels.ExpSquaredKernel(1.5)
gp = george.GP(kernel)
var=np.ones(len(Xin))
gp.compute(Xin,var)
x_pred = np.linspace(0, 200, 100)
pred, pred_var = gp.predict(data, x_pred, return_var=True)

plt.fill_between(x_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),color="k", alpha=0.2)
plt.plot(x_pred, pred, "k", lw=1.5, alpha=0.5)
plt.plot(Xin,data,"*")
plt.xlabel("x")
plt.ylabel("y");

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.2.1</span>

In the last lesson, when we applied a neural network to separate two samples, we cared only about minimizing the binary cross entropy. We did not care about knowing where the points are distributed. When we fit a function, we need to guess a form to fit the data and minimize $\chi^2$. Do we need to do that for a neural network?  

A) The neural network needs an architecture that is similar to the functional guess.\
B) If there are sufficient parameters, the neural network can approximate the function, no matter the form.\
C) Specific architectures are needed for specific problems (CNNs for images, RNNs for time series).\
D) The neural network needs to work with our statistical tools like Gaussian processes and f-tests to get the right function.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.2.2</span>

In the preceding code cell, `L15.2-runcell02`, we used the Gaussian Process regression algorithm to fit a curve to some data. Explore using different metrics in the function <a href="https://dfm.io/george/dev/user/kernels/#george.kernels.ExpSquaredKernel" target="_blank">`george.kernels.ExpSquaredKernel`</a> (currently, the default is set to 1.5). The code cell is repeated below, in the notebook. 

What approximate value of the metric should one use to smooth out the extraneous wiggles in the plot?

A) 0.5\
B) 2.5\
C) 5\
D) 20\
E) 100

In [ ]:
#>>>EXERCISE: L15.2.2

#now let's run GP on this guy

kernel = np.var(data) * kernels.ExpSquaredKernel(1.5)
gp = george.GP(kernel)
var=np.ones(len(Xin))
gp.compute(Xin,var)
x_pred = np.linspace(0, 200, 100)
pred, pred_var = gp.predict(data, x_pred, return_var=True)

plt.fill_between(x_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),color="k", alpha=0.2)
plt.plot(x_pred, pred, "k", lw=1.5, alpha=0.5)
plt.plot(Xin,data,"*")
plt.xlabel("x")
plt.ylabel("y");

<a name='section_15_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L15.3 An Example with PyTorch: Fitting a Parabola</h2>  

| [Top](#section_15_0) | [Previous Section](#section_15_2) | [Exercises](#exercises_15_3) | [Next Section](#section_15_4) |


In [ ]:
#>>>RUN: L15.3-runcell01

torch.manual_seed(1)    # reproducible

x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
y = x.pow(2) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)

# view data
plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis')
plt.xlabel('Independent variable')
plt.ylabel('Dependent variable')
plt.show()


In [ ]:
#>>>RUN: L15.3-runcell02

torch.manual_seed(1)    # reproducible

# this is one way to define a network
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

net = Net(n_feature=1, n_hidden=10, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)#stochastic Gradient Descent
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

In [ ]:
#>>>RUN: L15.3-runcell03

def makePlot(x,y,prediction,ax,fig,images,t,loss,ymin,ymax):
    # plot and show learning process
    plt.cla()
    ax.set_title('Regression Analysis', fontsize=35)
    ax.set_xlabel('Independent variable', fontsize=24)
    ax.set_ylabel('Dependent variable', fontsize=24)
    ax.set_ylim(ymin,ymax)
    ax.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
    ax.plot(x.data.numpy(), prediction.data.numpy(), 'g-', lw=3)
    ax.text(0.6, 0.7, 'Epoch = %d' % t, fontdict={'size': 24, 'color':  'red'})
    ax.text(0.6, 0.3, 'Loss = %.4f' % loss.data.numpy(),fontdict={'size': 24, 'color':  'red'}) 
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)

def train(x,y,net,loss_func,opt,nepochs,ymin,ymax):
    images = []
    fig, ax = plt.subplots(figsize=(12,7))
    for epoch in range(nepochs):
        if epoch % 50 == 0: 
            print("epoch:",epoch)
        prediction = net(x)
        loss = loss_func(prediction, y) 
        opt.zero_grad()
        loss.backward() 
        optimizer.step()
        nplots = int(nepochs/40)
        if epoch % nplots == 0:
            makePlot(x,y,prediction,ax,fig,images,epoch,loss,ymin,ymax)
    return images
    
from IPython.display import Image
images=train(x,y,net,loss_func,optimizer,200,-0.1,1.5)
imageio.mimsave('data/L15/curve_1.gif', images, fps=10)
Image(open('data/L15/curve_1.gif','rb').read())

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.3.1</span>

What are the dimensions of the input, output, and hidden layers? Enter your answer as a list of integers: `[input, output, hidden]`

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.3.2</span>

Why do we choose to use mean squared error for our loss function? Select all the apply:

A) This is the same as our original fit minimization (least squares).\
B) We did not give uncertainties, so we cannot do chi2.\
C) Mean squared error is the closest we get to absolute value.\
D) Actually, we should not use a NN at all, and we really should do a full systematic approach of adding polynomials and fitting to determine the functional form.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.3.3</span>

You saw that increasing the number of hidden parameters (i.e. the number of neurons in the hidden layer) gave a better by-eye fit. Why not use as many hidden parameters as possible? Wouldn't this make an even better fit to the data? Why would it NOT be ideal to arbitrarily add more hidden parameters? Select all that apply:

A) It can lead to overfitting of the model to the training data.\
B) It can make the model too simple and unable to learn complex features.\
C) It can require more computational resources.\
D) It can decrease the accuracy of the model on the training data.\
E) There is an ideal number of hidden parameters that works universally well for all neural networks.


<a name='section_15_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L15.4 Another Example: Fitting a Sine Function</h2>  

| [Top](#section_15_0) | [Previous Section](#section_15_3) | [Exercises](#exercises_15_4) | [Next Section](#section_15_5) |


In [ ]:
#>>>RUN: L15.4-runcell01

torch.manual_seed(1)    # reproducible
x = torch.unsqueeze(torch.linspace(-10, 10, 1000), dim=1)  # x data (tensor), shape=(100, 1)
y = torch.sin(x) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis')
plt.xlabel('Independent variable')
plt.ylabel('Dependent variable')
plt.show()

In [ ]:
#>>>RUN: L15.4-runcell02

#redefine network
net = Net(n_feature=1, n_hidden=10, n_output=1)     # define the network, try changing to n_hidden=100
#net = Net(n_feature=1, n_hidden=100, n_output=1)

images=train(x,y,net,loss_func,optimizer,200,-1.1,1.1)
imageio.mimsave('data/L15/curve_2.gif', images, fps=10)
Image(open('data/L15/curve_2.gif','rb').read())

In [ ]:
#>>>RUN: L15.4-runcell03

torch.manual_seed(1)    # reproducible

# another way to define a network
net = torch.nn.Sequential(
        torch.nn.Linear(1, 100),
        torch.nn.Linear(100, 100),
        torch.nn.Linear(100, 1),
    )
print(net[0].weight[0:10])
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
images=train(x,y,net,loss_func,optimizer,200,-1.1,1.1)
imageio.mimsave('data/L15/curve_2.gif', images, fps=10)
Image(open('data/L15/curve_2.gif','rb').read())

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.4.1</span>

Which of the following statements describes why linear activation function layers are not suitable for classification of data which is strongly nonlinear?

A) A linear activation function cannot capture complex nonlinear relationships between the input features and the output classes.\
B) A linear activation function is only effective for classification tasks with linearly separable data.\
C) A linear activation function is prone to underfitting and can lead to poor performance on the training and test sets.\
D) A linear activation function can introduce too much noise into the model and reduce its accuracy.

<a name='section_15_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L15.5 Sine Function Continued: Adjusting the Network</h2>  

| [Top](#section_15_0) | [Previous Section](#section_15_4) | [Exercises](#exercises_15_5) |

In [ ]:
#>>>RUN: L15.5-runcell01

torch.manual_seed(1)    # reproducible

# another way to define a network
net = torch.nn.Sequential(
        torch.nn.Linear(1, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.Linear(50, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
images=train(x,y,net,loss_func,optimizer,200,-1.1,1.1)
imageio.mimsave('data/L15/curve_2.gif', images, fps=10)
Image(open('data/L15/curve_2.gif','rb').read())

In [ ]:
#>>>RUN: L15.5-runcell02

torch.manual_seed(1)    # reproducible

# another way to define a network
net = torch.nn.Sequential(
        torch.nn.Linear(1, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
images=train(x,y,net,loss_func,optimizer,400,-1.1,1.1)
imageio.mimsave('data/L15/curve_2.gif', images, fps=10)
Image(open('data/L15/curve_2.gif','rb').read())

In [ ]:
#>>>RUN: L15.5-runcell03

torch.manual_seed(1)    # reproducible

net = torch.nn.Sequential(
        torch.nn.Linear(1, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
images=train(x,y,net,loss_func,optimizer,400,-1.1,1.1)
imageio.mimsave('data/L15/curve_2.gif', images, fps=10)
Image(open('data/L15/curve_2.gif','rb').read())

In [ ]:
#>>>RUN: L15.5-runcell04

torch.manual_seed(1)    # reproducible

net = torch.nn.Sequential(
        torch.nn.Linear(1, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
images=train(x,y,net,loss_func,optimizer,400,-1.1,1.1)
imageio.mimsave('data/L15/curve_2.gif', images, fps=10)
Image(open('data/L15/curve_2.gif','rb').read())

In [ ]:
#>>>RUN: L15.5-runcell05

torch.manual_seed(1)    # reproducible

x=torch.unsqueeze(torch.linspace(0, 200, 201), dim=1) 
y=torch.from_numpy(data.reshape(len(data),1).astype('float32'))

# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)

# view data
plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis')
plt.xlabel('Independent variable')
plt.ylabel('Dependent variable')
plt.show()


In [ ]:
#>>>RUN: L15.5-runcell06

torch.manual_seed(1)    # reproducible

def makePlot(x,y,prediction,ax,fig,images,t,loss,ymin,ymax):
    # plot and show learning process
    plt.cla()
    ax.set_title('Regression Analysis', fontsize=35)
    ax.set_xlabel('Independent variable', fontsize=24)
    ax.set_ylabel('Dependent variable', fontsize=24)
    ax.set_ylim(ymin,ymax)
    ax.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
    ax.plot(x.data.numpy(), prediction.data.numpy(), 'g-', lw=3)
    ax.text(125, 16, 'Epoch = %d' % t, fontdict={'size': 24, 'color':  'red'})
    ax.text(125, 14, 'Loss = %.4f' % loss.data.numpy(),fontdict={'size': 24, 'color':  'red'}) 
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)


net = torch.nn.Sequential(
        torch.nn.Linear(1, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
images=train(x,y,net,loss_func,optimizer,300,5,20)
imageio.mimsave('data/L15/curve_gaus.gif', images, fps=10)
Image(open('data/L15/curve_gaus.gif','rb').read())

In [ ]:
#>>>RUN: L15.5-runcell07

torch.manual_seed(1)    # reproducible

n_hidden=10
net = torch.nn.Sequential(
        torch.nn.Linear(1, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
images=train(x,y,net,loss_func,optimizer,1000,5,20)
imageio.mimsave('data/L15/curve_gaus.gif', images, fps=10)
Image(open('data/L15/curve_gaus.gif','rb').read())

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.5.1</span>

Consider the following neural network. How many hidden layers are there in this model? Enter your answer as an integer.

<pre>
net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 1),
    )
</pre>


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 15.5.2</span>

Run the code cell shown below multiple times, **varying the number of hidden layers each time.** Which of the following statements describes the effect of increasing the number of hidden layers? Select all that apply:

A) Increasing the number of hidden layers always leads to better classification performance.\
B) Increasing the number of hidden layers can help the model learn more complex patterns and improve classification performance, but may also increase the risk of overfitting.\
C) Increasing the number of hidden layers increases computational load and should only be done if better performance is needed.\
D) Increasing the number of hidden layers is only beneficial if the number of neurons in each layer is also increased.

**Extra:** Try also varying the number of parameters within the hidden layers.

In [ ]:
#>>>EXERCISE: L15.5.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def train(x,y,net,loss_func,opt,nepochs,ymin,ymax):
    images = []
    fig, ax = plt.subplots(figsize=(12,7))
    for epoch in range(nepochs):
        if epoch % 200 == 0: 
            print("epoch:",epoch)
        prediction = net(x)
        loss = loss_func(prediction, y) 
        opt.zero_grad()
        loss.backward() 
        optimizer.step()
        # Minimize plots for faster running
        if epoch == nepochs-1:
            makePlot(x,y,prediction,ax,fig,images,epoch,loss,ymin,ymax)
    return images

torch.manual_seed(1)    # reproducible

n_hidden=30

#VARY THE NUMBER OF HIDDEN LAYERS BELOW
net = torch.nn.Sequential(
        torch.nn.Linear(1, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, 1),
    )
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
images=train(x,y,net,loss_func,optimizer,1000,5,20)
imageio.mimsave('data/L15/curve_gaus.gif', images, fps=10)
Image(open('data/L15/curve_gaus.gif','rb').read())
